In [20]:
#%pip install -U fastmcp

<div class="alert alert-block alert-info" style="background-color: #05299E; color: white;">
    <h1 style="color: white;">Openai Agent Client</h1>
    <h4 style="color: white;">Cross-framework test for Openai Agents <<\MCP>> OpenAI Agents</h4>
    <h6 style="color: white;">@author: Cher Han</h6>
</div>

In [21]:
import asyncio
import json 
import nest_asyncio
import operator
import os
import sys
from dataclasses import dataclass, field

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from pydantic import BaseModel, Field
from typing import Annotated, Any, Dict, List, Tuple, TypedDict, Optional
from dotenv import load_dotenv



In [22]:

from agents import Agent, Runner, gen_trace_id, trace
from agents.mcp import MCPServer, MCPServerSse
from agents.model_settings import ModelSettings

In [23]:
from agents import (
    Agent,
    HandoffOutputItem,
    ItemHelpers,
    MessageOutputItem,
    RunContextWrapper,
    Runner,
    ToolCallItem,
    ToolCallOutputItem,
    TResponseInputItem,
    function_tool,
    handoff,
    trace,
)

In [24]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_MCP_HHAI_ENDPOINT = os.getenv("AZURE_MCP_HHAI_ENDPOINT")
AZURE_MCP_HHAI_API_KEY = os.getenv("AZURE_MCP_HHAI_API_KEY")

In [25]:
nest_asyncio.apply()

In [27]:
@dataclass
class ContextInfo:
    auth_header: Optional[dict] = None
    data_type: Optional[str] = None
    date: str = ""  # str(date.today())
    restart: bool = False
    last_agent_name: Optional[str] = None

async def main():
    server1 = MCPServerSse(
        name="appointmentWorkflow",
        params={
            "url": "http://localhost:8000/sse",
             "timeout":30
        },
        client_session_timeout_seconds=30,
    ) 
    hhai_mcp_server = MCPServerSse(
        name="healthhub_general_information",
        params={
            "url": f"{AZURE_MCP_HHAI_ENDPOINT}/sse",
            "headers": {"x-api-key": AZURE_MCP_HHAI_API_KEY},
        },
        cache_tools_list=True,
    )
    try:
        await server1.connect()
        await hhai_mcp_server.connect()
        wrapper = RunContextWrapper(
            context=ContextInfo(
                auth_header={
                    "Authorization": f"Bearer XXX",
                    "Content-Type": "application/json",
                }
            )
        )

        def dynamic_instructions(
            context: RunContextWrapper[ContextInfo],agent: Agent[ContextInfo]
        ) -> str:
            return f"""You help the user complete tasks using the connected tool. 
            Connect to hhai_mcp_server to get general information and server1 for appointment/booking workflow. 
            The last_agent_name is {context.context.context.last_agent_name}.
            The user ID is 
            Provide the tool with the last agent name if part of the parameterto get the correct response.
            """
        agent = Agent[ContextInfo](
            name="MCP Agent",
            instructions= dynamic_instructions,
            mcp_servers=[server1,hhai_mcp_server],
            model_settings=ModelSettings(tool_choice="required")
        )
        
        print("Chatbot started! Type 'exit' to quit.")
        last_agent_name = "appointment_agent"
        while True:
            
            message = input("\nYou: ")
            if message.lower() == 'exit':
                print("Goodbye!")
                break
            inputs: list[TResponseInputItem] = [{"content": message, "role": "user"}]
            print("wrapper",wrapper)
            result = Runner.run_streamed(agent,input=inputs, context=wrapper )
            async for event in result.stream_events():
                # We'll ignore the raw responses event deltas
                if event.type == "raw_response_event":
                    continue
                # When the agent updates, print that
                elif event.type == "agent_updated_stream_event":
                    print(f"Agent updated: {event.new_agent.name}")
                    continue
                # When items are generated, print them
                elif event.type == "run_item_stream_event":
                    if event.item.type == "tool_call_item":
                        print("-- Tool was called")
                    elif event.item.type == "tool_call_output_item":
                        print(f"-- Tool output: {event.item.output}")
                        try:
                            output = json.loads(event.item.output)
                            output = json.loads(output["text"])
                            last_agent_name = output["last_agent_name"]
                            if "last_agent_name" in output:
                                last_agent_name = output["last_agent_name"]
                                wrapper.context.last_agent_name = last_agent_name
                                print(f"Last agent name updated to: {last_agent_name}")
                        except json.JSONDecodeError:
                            pass
                    elif event.item.type == "message_output_item":
                        print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
                    else:
                        pass  # Ignore other event types
            inputs = result.to_input_list()
 

    finally:
        await server1.cleanup()
        #await hhai_mcp_server.cleanup()

In [28]:
if __name__ == "__main__":
    asyncio.run(main())

Chatbot started! Type 'exit' to quit.
wrapper RunContextWrapper(context=ContextInfo(auth_header={'Authorization': 'Bearer XXX', 'Content-Type': 'application/json'}, data_type=None, date='', restart=False, last_agent_name=None), usage=Usage(requests=0, input_tokens=0, output_tokens=0, total_tokens=0))
Agent updated: MCP Agent
-- Tool was called
-- Tool output: {"type":"text","text":"{\n  \"user_query\": \"I want to book an influenza vaccine\",\n  \"agent_final_response\": \"You already have an upcoming appointment for the Influenza vaccine on March 1, 2025, at 10:00 AM at Polyclinic A. \\n\\nWould you like to:\\n- Change the date, time, or location?\\n- Cancel the existing appointment?\\n- Keep the existing appointment?\",\n  \"status\": \"success\",\n  \"last_agent_name\": \"double_booking_check_agent\"\n}","annotations":null}
Last agent name updated to: double_booking_check_agent
-- Message output:
 You already have an upcoming appointment for the Influenza vaccine on March 1, 2025, a

Error cleaning up server: Attempted to exit a cancel scope that isn't the current tasks's current cancel scope


Goodbye!
